# Generating Python Code with Granite

**NOTE:** This recipe demonstrates how to use Granite Models to generate Python code from text prompts.

### Setting Up

**Install dependencies**

In [ ]:
%pip install "git+https://github.com/ibm-granite-community/utils" \
    langchain_community \
    transformers \
    replicate

**Select a model**

In [ ]:
from ibm_granite_community.notebook_utils import get_env_var
from langchain_community.llms import Replicate
from transformers import AutoTokenizer

model_path="ibm-granite/granite-3.3-8b-instruct"

model = Replicate(
    model=model_path,
    replicate_api_token=get_env_var('REPLICATE_API_TOKEN'),
)

tokenizer = AutoTokenizer.from_pretrained(model_path)

### One-shot Prompting with Granite

In One-shot prompting, you provide the model with a question and no examples. The model will generate an answer based on its training.

In [ ]:
from textwrap import dedent
from ibm_granite_community.langchain import TokenizerChatPromptTemplate

prompt = TokenizerChatPromptTemplate.from_template(dedent("""\
    Generate the Fibonacci sequence up to n terms.
    Used in Georgia Tech's CS 1301 course demonstrations.
    Args:
        n: Number of terms to generate
    Returns:
        List containing the Fibonacci sequence
    Raises:
        ValueError: If n is negative
"""),
tokenizer=tokenizer).format()

response = model.invoke(prompt)
print(response)

### Try the code

Copy and paste the generated code here to test it:

In [ ]:
# Paste your code here!
def fibonacci(n):
 if n < 0:
    raise ValueError("n must be a non-negative integer")
 elif n == 0:
    return []
 elif n == 1:
    return [0]
 else:
    fib_seq = [0, 1]

 for i in range(2, n):
    fib_seq.append(fib_seq[i-1] + fib_seq[i-2])
 return fib_seq

# Test the function
print(fibonacci(10))

### Few-shot Prompting with Granite

In few-shot prompting, you provide the model with a question and some examples to help it understand the pattern you want.

**Provide a list of Q&A examples**

In [ ]:
examples = [
    {
        "question": "Write a function to calculate the GPA based on Georgia Tech's 4.0 scale",
        "answer": """def calculate_gt_gpa(grades: list[str]) -> float:
    \"\"\"
    Calculate GPA based on Georgia Tech's 4.0 scale.

    Args:
        grades: List of letter grades (A, B, C, D, F)

    Returns:
        The calculated GPA on a 4.0 scale

    Raises:
        ValueError: If an invalid grade is provided
    \"\"\"
    grade_points = {'A': 4.0, 'B': 3.0, 'C': 2.0, 'D': 1.0, 'F': 0.0}

    total_points = 0
    for grade in grades:
        if grade not in grade_points:
            raise ValueError(f"Invalid grade: {grade}")
        total_points += grade_points[grade]

    return total_points / len(grades) if grades else 0.0"""
    },
    {
        "question": "Create a function to convert between Celsius and Fahrenheit for GT lab experiments",
        "answer": """def convert_temperature(temp: float, to_unit: str) -> float:
    \"\"\"
    Convert temperature between Celsius and Fahrenheit for GT lab experiments.

    Args:
        temp: Temperature value to convert
        to_unit: Target unit ('C' for Celsius, 'F' for Fahrenheit)

    Returns:
        Converted temperature value

    Raises:
        ValueError: If to_unit is not 'C' or 'F'
    \"\"\"
    if to_unit not in ['C', 'F']:
        raise ValueError("Unit must be 'C' or 'F'")

    if to_unit == 'C':
        return (temp - 32) * 5/9
    else:
        return (temp * 9/5) + 32"""
    }
]

**Assemble the prompt template**

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{question}"),
        ("ai", "{answer}"),
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

chat_template = TokenizerChatPromptTemplate.from_messages(
    [
        few_shot_prompt,
        ("human", "{question}"),
    ],
    tokenizer=tokenizer
)

print(chat_template.input_variables)

**View the completed prompt**

In [ ]:
prompt = "Write a Python class for a Georgia Tech Credit Union account with methods for deposit, withdraw, check balance, and track transactions. Include a Buzz Card account type option."

print(chat_template.format(question=prompt))

**Run the model**

In [ ]:
chain = chat_template | model
response = chain.invoke({"question": prompt})
print(response)

### Adding a System Prompt

A system prompt provides additional instructions and context for all queries.

In [ ]:
from langchain_core.messages import SystemMessage

system_prompt = SystemMessage(content=dedent("""\
    You are a Python expert from Georgia Tech's College of Computing who writes clean,
    efficient, and well-documented code. Follow PEP 8 style guidelines and Georgia Tech's
    computational thinking principles.
    Always include type hints and comprehensive docstrings. Only output Python code, no explanations.
"""))

chat_template_with_system_message = TokenizerChatPromptTemplate.from_messages(
    [
        system_prompt,
        few_shot_prompt,
        ("human", "{question}"),
    ],
    tokenizer=tokenizer
)

print(chat_template_with_system_message.input_variables)

**Run the model with system prompt**

In [ ]:
chain_sys = chat_template_with_system_message | model
response = chain_sys.invoke({"question": prompt})
print(response)

Copy and paste the generated code here to test it:

In [ ]:
# Paste the generated response here and run it!
class BankAccount:
    def __init__(self, initial_balance: float = 0.0):
        """
        BankAccount class for managing bank account transactions.

        Args:
            initial_balance: Initial balance for the account (default: 0.0)
        """
        self.balance = initial_balance

    def deposit(self, amount: float) -> None:
        """
        Deposit amount into the bank account.

        Args:
            amount: Amount to deposit into the account
        """
        self.balance += amount

    def withdraw(self, amount: float) -> None:
        """
        Withdraw amount from the bank account.

        Args:
            amount: Amount to withdraw from the account

        Raises:
            ValueError: If withdrawal would result in a negative balance
        """
        if self.balance - amount < 0:
            raise ValueError("Insufficient funds")
        self.balance -= amount

    def check_balance(self) -> float:
        """
        Check the current balance of the bank account.

        Returns:
            Current balance of the account
        """
        return self.balance

# Test the BankAccount class
if __name__ == "__main__":
    account = BankAccount(100.0)
    print(account)

    account.deposit(50)
    print(f"Balance after deposit: ${account.check_balance():.2f}")

    account.withdraw(30)
    print(f"Balance after withdrawal: ${account.check_balance():.2f}")

### For further study

- Try different queries and test if the model generates correct Python code
- Experiment with more complex programming tasks like data processing or algorithm implementation
- Add more diverse examples to the `examples` list and see how it affects the outputs
- Try generating different types of Python code (OOP vs. functional style)